## Things you need
+ config file with api key
+ Your user agent info can get https://www.whatismybrowser.com/detect/what-is-my-user-agent and replace the headers variable with your info
    + header variable is line 10 in both scraping block
+ Script splits up your sources into 2 queries and saves them as csv files.  I created a csv folder for them

## change query
Hit other candidates as well and make sure that the range (20,31) is no more than 30 days ago.  If you get a out of range error just change 20 to whatever day of month it is, or day of month +1

In [16]:
import pandas as pd
import config
import math
import csv
import sys
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)


## global variables
candidates_list = []
total_count = []


def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Cand_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Cand_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Cand_df[Cand_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))

def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

def hit_api(start, end, q, myString2):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myString2,
                                          language='en',
                                          from_param='2019-09-{}'.format(
                                              start_str),
                                          to='2019-09-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)


In [17]:

## import data dictionary form github
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)

# picking data sources from file on github
a= df.iloc[[16,17,24],:]
b= df.iloc[[40,43,77],:]
list_sources = a["sources"].tolist()

# build out string for query request
myString = ",".join(list_sources)

list_sources2 = b["sources"].tolist()
myString2 = ",".join(list_sources2)



## Scrape Wall street journal and NYT

In [19]:
for x in range(20, 31):
    df = hit_api(x, x+1, 'Biden OR Joe Biden', myString2)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 48
query will return: 36
query will return: 82
query will return: 130
query will return: 145
query will return: 149
query will return: 121
query will return: 72
query will return: 37
query will return: 53
query will return: 656
we should expect: 3058 articles
we have: 1656 articles
1402 results came into query that exceeded 100 hits in a day.
we have 521 unique links
we have duplciate shape df of: (1135, 7) 
1655
1654
1653
1652
1651
1650
1649
1648
1647
1646
1645
1644
1643
1642
no words found
1641
1640
1639
1638
1637
1636
1635
1634
1633
no words found
1632
1631
1630
1629
1628
1627
1626
no words found
1625
1624
1623
1622
1621
1620
1619
1618
1617
1616
1615
1614
1613
1612
1611
1610
1609
1608
1607
1606
1605
1604
1603
1602
1601
1600
1599
1598
1597
1596
1595
1594
1593
1592
1591
1590
1589
1588
1587
1586
1585
1584
1583
1582
1581
1580
1579
1578
1577
1576
1575
1574
1573
1572
1571
1570
1569
1568
1567
1566
1565
1564
1563
1562
1561
1560
1559
1558
1557
1556
1555
1554
1553
1552
1551

153
152
151
150
149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
no words found
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
no words found
6
5
4
3
2
1
0


## Scrape WAPO New York magazine


In [14]:
for x in range(20, 31):
    df = hit_api(x, x+1, 'Biden OR Joe Biden', myString)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 40
query will return: 24
query will return: 47
query will return: 78
query will return: 108
query will return: 117
query will return: 93
query will return: 53
query will return: 24
query will return: 39
query will return: 525
we should expect: 1611 articles
we have: 1044 articles
567 results came into query that exceeded 100 hits in a day.
we have 649 unique links
we have duplciate shape df of: (395, 7) 
1043
1042
1041
1040
1039
1038
1037
1036
1035
1034
1033
1032
1031
1030
1029
1028
1027
1026
1025
1024
1023
1022
1021
1020
1019
1018
1017
1016
1015
1014
1013
1012
1011
1010
1009
1008
1007
1006
1005
1004
1003
1002
1001
1000
999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
9

,author,content,description,publishedAt,source,title,url,full_art
0,Margot Sanger-Katz and Sydney Ember,"Were addressing it on both ends, Mr. Sanders s...","On top of his “Medicare for all” proposal, Mr....",2019-09-21,The New York Times,Bernie Sanders Calls for Eliminating Americans...,https://www.nytimes.com/2019/09/21/us/politics...,"“We’re addressing it on both ends,” Mr. Sander..."
1,Emily Badger,Mr. Sanders has similarly proposed a tax on em...,A plan for national rent control and for reini...,2019-09-21,The New York Times,Should People Profit From Housing? Bernie Sand...,https://www.nytimes.com/2019/09/19/upshot/Bern...,Mr. Sanders has similarly proposed a tax on em...
2,Sydney Ember,But she also said she was interested in Ms. Wa...,The Vermont senator benefited from a wave of e...,2019-09-20,The New York Times,"Young Voters Still ‘Feel the Bern,’ but Not Ju...",https://www.nytimes.com/2019/09/20/us/politics...,As he fights anew for the Democratic nominatio...
3,Jennifer Finney Boylan,Another loser last night was Senator Bernie Sa...,Senators Warren and Booker shone in a forum fo...,2019-09-21,The New York Times,Love in the Time of Democrats,https://www.nytimes.com/2019/09/21/opinion/eli...,"Meanwhile, as an older American — and one from..."
4,Astead W. Herndon,"Jaylan Jones, 20, said, Older people have that...",An organic effort by black millennials and Gen...,2019-09-20,The New York Times,Younger Black Voters to Their Parents: Break U...,https://www.nytimes.com/2019/09/20/us/politics...,“Young people are constantly signaling what’s ...
5,Lisa Lerer and Reid J. Epstein,Im going to keep the focus on the fact that Do...,Reports that President Trump sought help from ...,2019-09-21,The New York Times,"For Biden, Whistle-Blower Complaint Could Cut ...",https://www.nytimes.com/2019/09/21/us/politics...,Though he has yet to call for impeachment proc...
6,Matt Stevens and Maggie Astor,The tragic story of the Triangle factory fire ...,Here are six things that happened on the presi...,2019-09-21,The New York Times,"Bill de Blasio Bows Out, Elizabeth Warren’s Bi...",https://www.nytimes.com/2019/09/21/us/politics...,“The tragic story of the Triangle factory fire...
7,"Eliza Collins, John McCormick",Kamala Harris\r\ns campaign says she is focusi...,Kamala Harris’s Iowa Strategy Is Well-Worn—and...,2019-09-20,The Wall Street Journal,Kamala Harris’s Iowa Strategy Is Well-Worn—and...,https://www.wsj.com/articles/kamala-harriss-io...,Kamala Harris’s campaign says she is focusing ...
8,Eliza Collins,"BURLINGTON, Vt.\r\nBernie Sanders\r\nsays that...",Sanders's 'Revolution' in Burlington: Working ...,2019-09-20,The Wall Street Journal,Sanders's 'Revolution' in Burlington: Working ...,https://www.wsj.com/articles/sanderss-revoluti...,"BURLINGTON, Vt.—Bernie Sanders says that if he..."
9,None,The Republicans are highly successful at getti...,We asked readers what topics have been slighte...,2019-09-21,The New York Times,The Next Debate: Let’s Talk About …,https://www.nytimes.com/2019/09/21/opinion/let...,"To the Editor:\n\nSo far, so-called “women’s i..."
